In [1]:
import sys
import os
import pandas as pd
import spacy
import ast

%load_ext autoreload
%autoreload 2
PATH = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(PATH)

In [2]:
data = pd.read_csv(PATH + '/skills_matcher/data/data_final_raw.csv')
data.head(2)

,Unnamed: 0,ISCO,major_job,job,position,location,description,entities_auto_label,entities_manual_label
0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': physics, 'label': 'KNOWLEDGE'}, {'..."
1,1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': RICS, 'label': 'SKILL'}, {'entity'..."


In [64]:
def extract_skills_auto(s):
    words_pattern = '[a-z]+'
    return re.findall(words_pattern, s, flags=re.IGNORECASE)

In [79]:
skills_auto = data['entities_auto_label'].apply(extract_skills_auto)
skills_auto

0                     []
1                     []
2                     []
3       [Python, Python]
4       [Python, Python]
              ...       
3502                  []
3503                  []
3504                  []
3505                  []
3506                  []
Name: entities_auto_label, Length: 3507, dtype: object

In [129]:
import re

#  new with non word char inside entities
def extract_entities_2(s):
    pattern = re.compile("'entity': [a-zA-Z]+\W*([a-zA-Z]|\s)*")
    res = re.finditer(pattern, s)
    return [ mo.group().split(",")[0].strip().lstrip("'entity': ") for mo in res if mo.group() != ',']

def extract_lables(s):
    pattern = re.compile("'label':\s*'\w+'")
    res = re.finditer(pattern, s)
    return [ mo.group().partition("': '")[2].strip("'") for mo in res]

In [32]:
labels = data['entities_manual_label'].apply(extract_lables)
labels

0                               [KNOWLEDGE, SKILL, SKILL]
1                                   [SKILL, SKILL, SKILL]
2                                      [SKILL, KNOWLEDGE]
3       [SKILL, KNOWLEDGE, KNOWLEDGE, KNOWLEDGE, SKILL...
4                                                      []
                              ...                        
3502                                          [KNOWLEDGE]
3503                                          [KNOWLEDGE]
3504                                          [KNOWLEDGE]
3505                                          [KNOWLEDGE]
3506                                          [KNOWLEDGE]
Name: entities_manual_label, Length: 3507, dtype: object

In [33]:
data.entities_manual_label[3505]

"[{'entity': driver's license, 'label': 'KNOWLEDGE'}]"

In [35]:
entities = data['entities_manual_label'].apply(extract_entities_2)
entities

0                              [physics, English, German]
1                           [RICS, CABE, driving licence]
2                                           [CIS, Fawkes]
3       [Python, programming, High Energy Physics data...
4                                                      []
                              ...                        
3502                                    [driving licence]
3503                                   [driver's license]
3504                                   [driver's license]
3505                                   [driver's license]
3506                                   [driver's license]
Name: entities_manual_label, Length: 3507, dtype: object

In [28]:
list(zip(labels[0], entities[0]))

[('KNOWLEDGE', 'physics'), ('SKILL', 'English'), ('SKILL', 'German')]

In [38]:
print(labels[0])
print(entities[0])

['KNOWLEDGE', 'SKILL', 'SKILL']
['physics', 'English', 'German']


In [118]:
final_list = []

for num in range(len(data)):
    
    dict_ = {}
    
    for index, key in enumerate(entities[num]):
        if key not in dict_.keys():
            dict_[key] = labels[num][index]

    for index, value in enumerate(skills_auto[num]):
        if value not in dict_.keys():
            dict_[value] = 'SKILL'
            
    final_list.append(dict_)

final_list[3]

{'Python': 'SKILL',
 'programming': 'KNOWLEDGE',
 'High Energy Physics data analysis': 'KNOWLEDGE',
 'programming techniques': 'KNOWLEDGE',
 'English': 'SKILL',
 'French': 'SKILL'}

In [159]:
def get_dict(x):
    
    dict_ = {}
    
    entities = extract_entities_2(x['entities_manual_label'])
    labels = extract_lables(x['entities_manual_label'])
    for index, key in enumerate(entities):
        if key not in dict_.keys():
            dict_[key] = labels[index]

    skills_auto =extract_skills_auto(x['entities_auto_label'])
    for index, value in enumerate(skills_auto):
        if value not in dict_.keys():
            dict_[value] = 'SKILL'
    
    skills_list = []
    knowledge_list = []
    min_exp_list = []
    level_list = []
            
    for entity, label in dict_.items():
        if label == 'SKILL':
            skills_list.append(entity)
        if label == 'KNOWLEDGE':
            knowledge_list.append(entity)
        if label == 'MIN_EXP':
             min_exp_list.append(entity)
        if label == 'LEVEL':
            level_list.append(entity)
    
    x["SKILL"] = skills_list
    x["KNOWLEDGE"] = knowledge_list
    x["MIN_EXP"] = min_exp_list
    x["LEVEL"] = level_list
    
    return x

In [161]:
data_cleaned = data.apply(get_dict, axis=1)

In [157]:
data_cleaned.head()

,Unnamed: 0,ISCO,major_job,job,position,location,description,entities_auto_label,entities_manual_label,SKILL,KNOWLEDGE,MIN_EXP,LEVEL
0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': physics, 'label': 'KNOWLEDGE'}, {'...","[English, German]",[physics],[],[]
1,1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': RICS, 'label': 'SKILL'}, {'entity'...","[RICS, CABE, driving licence]",[],[],[]
2,2,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': CIS, 'label': 'SKILL'}, {'entity':...",[CIS],[Fawkes],[],[]
3,3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...,"(Python, Python)","[{'entity': Python, 'label': 'SKILL'}, {'entit...","[Python, English, French]","[programming, High Energy Physics data analysi...",[],[]
4,4,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...,"(Python, Python)",[],[Python],[],[],[]


In [147]:
data_cleaned['SKILL'][0]

['English', 'German']

In [ ]:
def print_final_table(index):
    list_skills = data_cleaned['SKILL'][index]
    list_values = 